In [1]:
import os
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [2]:
import google.generativeai as genai
from google.ai.generativelanguage_v1beta.types import content

key_gemini = os.environ["GOOGLE_API_KEY"]

genai.configure(api_key=key_gemini)

/workspaces/quiz_interview/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = genai.GenerativeModel("gemini-1.5-flash")
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "Hello"},
        {"role": "model", "parts": "Great to meet you. What would you like to know?"},
    ]
)
response = chat.send_message("I have 2 dogs in my house.")
print(response.text)
response = chat.send_message("How many paws are in my house?")
print(response.text)

That's wonderful!  What breeds are they?  Are they both pups, or do you have one of each?  Do they have any special tricks or personalities?  I'd love to hear more about them! 🐶 

Since you have two dogs, and each dog has four paws, there are a total of **eight** paws in your house! 🐾 



In [4]:
def retriever(query: str) -> list[dict[str, str]]:
    """ Search system that retrieves documents from a knowldgebase based in the text in the parameter 'query'."""
    return [
        {
            "id": "doc1",
            "content": "O acesso ao PJE é feito por meio do site do PJe ou do aplicativo PJe portable."
        },
        {
            "id": "doc2",
            "content": "O acesso ao PJE é feito por meio do site do PJe ou do aplicativo PJe portable."
        }
    ]

In [15]:
functions_dict = {
    "retriever": retriever
}

In [23]:
instruction = (
    "You are a helpful bot create to help user from the IT section called Setic in their daily activities\n"
    "You should help answer the user questions.\n"
    "you could use the functions provided to gather documents from other systems to aggregate relevant context.\n"
    "If the context provided by the functions is not relevant and you dont know the answer, just state this to the user and ask for clarification.\n"
    "Answer in the same user language.\n"
    # "In the beginning of the conversation you will not have any con"
    "If there isn't any context from a function called in the conversation yet, you should try to to use one of the functions to get some updated context.\n"
    )

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[retriever],
                              system_instruction=instruction)

In [24]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

In [27]:
chat = model.start_chat()

In [28]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("como faço para acessar o PJe?", tool_config=tool_config)
print(response.parts[0])
# chat.rewind();  # You are not actually calling the function, so remove this from the history.

text: "Para te ajudar a acessar o PJe, preciso entender melhor o que você precisa. Você pode me dizer qual tipo de acesso você precisa? Por exemplo: \n\n* Você precisa acessar o PJe como advogado? \n* Você precisa acessar o PJe como parte? \n* Você precisa acessar o PJe como servidor público?\n\nCom mais informações, posso te ajudar a encontrar a resposta que você precisa. \n"



In [19]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "retriever",
                  "args": {
                    "query": "acessar PJe"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CON

In [20]:
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")
        response = functions_dict[fn.name](**fn.args)

response     


retriever(query=acessar PJe)


[{'id': 'doc1',
  'content': 'O acesso ao PJE é feito por meio do site do PJe ou do aplicativo PJe portable.'},
 {'id': 'doc2',
  'content': 'O acesso ao PJE é feito por meio do site do PJe ou do aplicativo PJe portable.'}]

In [21]:
responses = {
    "retriever": response
}

In [22]:
# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(
        name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Para acessar o PJe, você pode usar o site do PJe ou o aplicativo PJe portable. 

